In [25]:
import pandas as pd
import numpy as np
import glob
import os
import os
import sys
sys.path.insert(0, '../')
#sys.path.insert(0, '../')
#sys.path.insert(0, './')
#from data.Reinhard import Reinhard
#sys.path.insert(0, './')
#from Reinhard import Reinhard
from models.model_mrcnn import _default_mrcnn_config, build_default
from visualization.explain import ExplainPredictions
import torch
import tqdm
from PIL import Image
import torchvision
from torchmetrics.classification import MulticlassConfusionMatrix
import torchvision.ops.boxes as bops
from torchmetrics.classification import Dice
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
model_input_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/models/eager-frog-489_mrcnn_model_100.pth"

In [4]:
test_config = dict(
    batch_size = 1,
    num_classes = 3
)

model_config = _default_mrcnn_config(num_classes=1 + test_config['num_classes']).config
model_amyb = build_default(model_config, im_size=1024)

In [5]:
model_amyb.load_state_dict(torch.load(model_input_path))

<All keys matched successfully>

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_amyb.eval().to(device)

GeneralizedRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(1024,), max_size=1024, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
         

In [7]:
def prepare_input(image):
    image_float_np = np.float32(image) / 255

    # define the torchvision image transforms
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
    ])

    input_tensor = transform(image)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_tensor = input_tensor.to(device)
    # Add a batch dimension:
    input_tensor = input_tensor.unsqueeze(0)
    return input_tensor, image_float_np

class_names = ['Cored', 'Diffuse', 'CAA']
def get_outputs(input_tensor, model, threshold):
    with torch.no_grad():
        # forward pass of the image through the modle
        outputs = model(input_tensor)
    #print(outputs)
    scores = list(outputs[0]['scores'].detach().cpu().numpy())
    # print("\n scores", max(scores))
    # index of those scores which are above a certain threshold
    thresholded_preds_inidices = [scores.index(i) for i in scores if i > threshold]
    #print(thresholded_preds_inidices)
    thresholded_preds_count = len(thresholded_preds_inidices)
    #print(thresholded_preds_count)
    #print("score before", scores)
    scores = np.array(scores)[thresholded_preds_inidices]
    #print("score after", scores)
    # get the masks
    masks = (outputs[0]['masks']>0.5).squeeze().detach().cpu().numpy()
    if len(masks.shape)==2:
        masks = np.array([masks])
    # discard masks for objects which are below threshold
    masks = masks[thresholded_preds_inidices]
    #print(masks.shape)
    # get the bounding boxes, in (x1, y1), (x2, y2) format
    boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in outputs[0]['boxes'].detach().cpu()]
    # discard bounding boxes below threshold value
    boxes = np.array(boxes)[thresholded_preds_inidices]
    # get the classes labels
    # print('labels', outputs[0]['labels'])
    #print(outputs[0]['labels'])
    #print(thresholded_preds_count)
    #print(outputs[0]['labels'])
    #print(outputs[0]['labels'])
    labels = [class_names[i-1] for i in outputs[0]['labels']]
    #labels = [i for i in outputs[0]['labels']]
    #print(labels)
    labels = labels[:thresholded_preds_count+1]
    return masks, boxes, labels, scores

In [8]:
def expand_mask(mask_path):
    mask = Image.open(mask_path).convert('P')   
    mask = np.array(mask)
    # instances are encoded as different colors
    obj_ids = np.unique(mask)
    # first id is the background, so remove it
    obj_ids = obj_ids[1:]
    # split the color-encoded mask into a set
    # of binary masks
    masks = mask == obj_ids[:, None, None]
    num_objs = len(obj_ids)
    
    boxes = []
    for i in range(num_objs):
        pos = np.where(masks[i])
        xmin = np.min(pos[1])
        xmax = np.max(pos[1])
        ymin = np.min(pos[0])
        ymax = np.max(pos[0])
        #print(pos)
        if xmax <= xmin and ymax <=ymin:
            print("degenrate boxes", mask_path)
            print(len(obj_ids))
            break
        boxes.append([xmin, ymin, xmax, ymax])
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    x = [id // 50 for id in obj_ids]
    #x = [1 for i in x if i==3]
    #x = [3 for i in x if i==4]
    labels = torch.tensor(x)
    masks = torch.as_tensor(masks, dtype=torch.uint8)
    #image_id = torch.tensor([idx])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    # suppose all instances are not crowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["masks"] = masks
    #target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd
    print(labels)
    return target

In [9]:
def match_mask(device,masked_image,binary_array):
    #num_classes = len(set(list(np.unique(masked_image)) +  list(np.unique(binary_array))))
    num_classes = 3
    metric = MulticlassConfusionMatrix(num_classes=num_classes).to(device)
    conf_final=torch.tensor(np.zeros((num_classes,num_classes))).to(device).to(torch.int64)
    for i in range(len(masked_image)):
        preds = torch.tensor(masked_image[i]).to(device).to(torch.int64)
        target = torch.tensor(binary_array[i]).to(device).to(torch.int64)
        a1 = metric(preds,target)
        conf_final = conf_final + a1     
    conf_final_np = conf_final.cpu().numpy()
    total_predicted = np.sum(conf_final_np, axis=0) 
    diag_elements = np.diag(conf_final_np)
    precision = diag_elements/total_predicted
    total_actual = np.sum(conf_final_np, axis=1) 
    recall = diag_elements/total_actual
    f1_score = (2*precision*recall)/(recall+precision)
    iou_coeff = (diag_elements)/(total_predicted+total_actual-diag_elements)
    
    #csv_filename_tosave = "Eval_Metric_"+ geofile.split(".")[0] + ".csv"
    eval_metrics = pd.DataFrame({"Class":[ 'Cored', 'Diffuse', 'CAA'],"Precision":precision, "recall":recall,"f1_score":f1_score,"iou_coeff":iou_coeff})
    #eval_metrics.to_csv(os.path.join(eval_dir,csv_filename_tosave))
    #return eval_metrics[eval_metrics["Class"]=="Pre/True"]["f1_score"].values[0]
    return eval_metrics

In [10]:
def match_label(pred_label, gt_label):
    if pred_label==gt_label:
        return True
    else:
        return False

def actual_label_target(gt_label):
    if (gt_label.cpu().numpy()==1) or (gt_label.cpu().numpy()==3):
        return "Cored"
    if (gt_label.cpu().numpy()==2):
        return "Diffuse"
    if (gt_label.cpu().numpy()==4):
        return "CAA"
    return None

In [11]:
test_folder  = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/"
image_folders = glob.glob(os.path.join(test_folder, '*'))
output_df =  pd.DataFrame()
#output_df = pd.DataFrame({"img_crop":[], "target_label":[], "pred_label":[], "target_mask_index":[],"pred_mask_index":[], "dice/f1_score":[],"matched_label":[],"image_folder":[]})
for image_folder in image_folders:
    images = glob.glob(os.path.join(image_folder, "images", "*.png"))
    masks = glob.glob(os.path.join(image_folder, "labels", "*.png"))
    images.sort()
    masks.sort()
    img_list = []
    target_mask_list= []
    pred_mask_list=[]
    f1_score_list = []
    matched_label_list = []
    actual_label_list = []
    pred_label_list =[]
    detection_score_list=[]
    for img, mask in zip(images,masks):
        #img_name = os.path.basename(img).split('.')[0]
        image = np.array(Image.open(img))
        #mask = np.array(Image.open(mask))
        target = expand_mask(mask)
        if image.shape[2] == 4:
            image = image[:,:, :3]
        input_tensor, image_float_np = prepare_input(image)
        masks, boxes, labels, scores = get_outputs(input_tensor, model_amyb, 0.2)
        #print(len(labels))
        for i in range(len(target['masks'])):
            for j in range(len(masks)):
                target_label = actual_label_target(target['labels'][i])
                actual_label_list.append(target_label)
                img_list.append(img)
                target_mask_list.append(i)
                #if len(masks[j].shape)>=2:
                    #print(target_label, labels[j])
                pred_label_list.append(labels[j])
                eval_metrics = match_mask(device, masks[j],target['masks'][i])
                #print(eval_metrics)
                matched_label = match_label(labels[j],target_label)
                #print(matched_label)
                f1_score = eval_metrics[eval_metrics["Class"]==target_label]["f1_score"].values[0]
                pred_mask_list.append(j)
                f1_score_list.append(f1_score)
                matched_label_list.append(matched_label)
                detection_score_list.append(scores[j])
            if len(masks)==0:
                target_label = actual_label_target(target['labels'][i])
                actual_label_list.append(target_label)
                img_list.append(img)
                target_mask_list.append(i)
                pred_label_list.append(None)
                pred_mask_list.append(None)
                f1_score_list.append(None)
                matched_label_list.append(None)
                detection_score_list.append(None)
    print(len(target_mask_list), len(pred_mask_list))            
    tmp = pd.DataFrame({"img_crop":img_list, "target_label":actual_label_list, "pred_label":pred_label_list, "target_mask_index":target_mask_list,"pred_mask_index":pred_mask_list, "dice/f1_score":f1_score_list,"matched_label":matched_label_list,"detection_score":detection_score_list})
    tmp["image_folder"] = image_folder
    if len(output_df)==0:
        output_df = tmp
    else:
        output_df= pd.concat([output_df,tmp],ignore_index=True)
       
         
        

tensor([2])


/tmp/ipykernel_280123/2329794364.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(binary_array[i]).to(device).to(torch.int64)
/tmp/ipykernel_280123/2329794364.py:14: RuntimeWarning: invalid value encountered in true_divide
  precision = diag_elements/total_predicted
/tmp/ipykernel_280123/2329794364.py:16: RuntimeWarning: invalid value encountered in true_divide
  recall = diag_elements/total_actual
/tmp/ipykernel_280123/2329794364.py:18: RuntimeWarning: invalid value encountered in true_divide
  iou_coeff = (diag_elements)/(total_predicted+total_actual-diag_elements)


tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([1])
tensor([1])
tensor([2])
tensor([1])


/tmp/ipykernel_280123/2329794364.py:17: RuntimeWarning: invalid value encountered in true_divide
  f1_score = (2*precision*recall)/(recall+precision)


tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([2])
tensor([1])
tensor([1])
tensor([2])
tensor([1])
tensor([3])
tensor([3])
tensor([3])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([3])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([3])
tensor([3])
tensor([2])
tensor([1])
tensor([2])
59 59
tensor([1])


/tmp/ipykernel_280123/2329794364.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(binary_array[i]).to(device).to(torch.int64)
/tmp/ipykernel_280123/2329794364.py:14: RuntimeWarning: invalid value encountered in true_divide
  precision = diag_elements/total_predicted
/tmp/ipykernel_280123/2329794364.py:16: RuntimeWarning: invalid value encountered in true_divide
  recall = diag_elements/total_actual
/tmp/ipykernel_280123/2329794364.py:18: RuntimeWarning: invalid value encountered in true_divide
  iou_coeff = (diag_elements)/(total_predicted+total_actual-diag_elements)


tensor([3])
tensor([1])
tensor([3])


/tmp/ipykernel_280123/2329794364.py:17: RuntimeWarning: invalid value encountered in true_divide
  f1_score = (2*precision*recall)/(recall+precision)


tensor([3])
tensor([3])
tensor([3])
tensor([3])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
13 13


In [13]:
output_df

,img_crop,target_label,pred_label,target_mask_index,pred_mask_index,dice/f1_score,matched_label,detection_score,image_folder
0,/gladstone/finkbeiner/steve/work/data/npsad_da...,Diffuse,Diffuse,0,0.0,0.911739,True,0.999552,/gladstone/finkbeiner/steve/work/data/npsad_da...
1,/gladstone/finkbeiner/steve/work/data/npsad_da...,Diffuse,Diffuse,0,0.0,0.775440,True,0.997847,/gladstone/finkbeiner/steve/work/data/npsad_da...
2,/gladstone/finkbeiner/steve/work/data/npsad_da...,Diffuse,None,0,NaN,NaN,None,NaN,/gladstone/finkbeiner/steve/work/data/npsad_da...
3,/gladstone/finkbeiner/steve/work/data/npsad_da...,Diffuse,Diffuse,0,0.0,0.912643,True,0.617667,/gladstone/finkbeiner/steve/work/data/npsad_da...
4,/gladstone/finkbeiner/steve/work/data/npsad_da...,Diffuse,None,0,NaN,NaN,None,NaN,/gladstone/finkbeiner/steve/work/data/npsad_da...
...,...,...,...,...,...,...,...,...,...
67,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,Cored,0,0.0,0.995646,True,0.999318,/gladstone/finkbeiner/steve/work/data/npsad_da...
68,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,Cored,0,0.0,0.994240,True,0.999396,/gladstone/finkbeiner/steve/work/data/npsad_da...
69,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,Cored,0,0.0,0.995676,True,0.998872,/gladstone/finkbeiner/steve/work/data/npsad_da...
70,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,Cored,0,0.0,0.997694,True,0.997166,/gladstone/finkbeiner/steve/work/data/npsad_da...


In [13]:
output_df["image_name"] =output_df["img_crop"].apply(lambda l:l.split("/")[-1])

In [27]:
output_df["pred_mask_index"].value_counts()

0.0    60
1.0     6
Name: pred_mask_index, dtype: int64

In [17]:
max_dice_pred = output_df.groupby(["img_crop","target_label","target_mask_index"])["dice/f1_score"].max().reset_index()

In [19]:
len(max_dice_pred)

66

In [20]:
matched_dice_pred_index = pd.merge(max_dice_pred,output_df, on = ["img_crop","target_label","target_mask_index","dice/f1_score"], how = "left")

In [21]:
matched_dice_pred_index

,img_crop,target_label,target_mask_index,dice/f1_score,pred_label,pred_mask_index,matched_label,detection_score,image_folder
0,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,0,0.997149,Cored,0.0,True,0.995691,/gladstone/finkbeiner/steve/work/data/npsad_da...
1,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,0,0.994890,Cored,0.0,True,0.999368,/gladstone/finkbeiner/steve/work/data/npsad_da...
2,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,0,0.987024,Cored,0.0,True,0.999254,/gladstone/finkbeiner/steve/work/data/npsad_da...
3,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,0,0.947496,Cored,0.0,True,0.998965,/gladstone/finkbeiner/steve/work/data/npsad_da...
4,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,0,0.994962,Cored,0.0,True,0.998806,/gladstone/finkbeiner/steve/work/data/npsad_da...
...,...,...,...,...,...,...,...,...,...
61,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,0,0.996804,Cored,0.0,True,0.999202,/gladstone/finkbeiner/steve/work/data/npsad_da...
62,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,0,0.998135,Cored,0.0,True,0.990439,/gladstone/finkbeiner/steve/work/data/npsad_da...
63,/gladstone/finkbeiner/steve/work/data/npsad_da...,Diffuse,0,NaN,None,NaN,None,NaN,/gladstone/finkbeiner/steve/work/data/npsad_da...
64,/gladstone/finkbeiner/steve/work/data/npsad_da...,Cored,0,0.998087,Cored,0.0,True,0.999560,/gladstone/finkbeiner/steve/work/data/npsad_da...


In [22]:
matched_dice_pred_index.groupby(["image_folder","target_label","pred_label"])["img_crop"].count()

image_folder                                                                                              target_label  pred_label
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json  Cored         Cored         12
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json  Cored         Cored         42
                                                                                                          Diffuse       Diffuse        6
Name: img_crop, dtype: int64

In [15]:
output_df.groupby(["image_folder","target_label","pred_label"])["img_crop"].count()

image_folder                                                                                              target_label  pred_label
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json  Cored         Cored         13
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json  Cored         Cored         45
                                                                                                                        Diffuse        1
                                                                                                          Diffuse       Cored          1
                                                                                                                        Diffuse        6
Name: img_crop, dtype: int64

In [16]:
output_df.groupby(["image_folder"])["dice/f1_score"].count()

image_folder
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json    13
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json    52
Name: dice/f1_score, dtype: int64

In [17]:
output_df[~output_df["dice/f1_score"].isna()].groupby(["image_folder"])["matched_label"].sum()

image_folder
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json    13
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json    51
Name: matched_label, dtype: object

In [18]:
output_df.groupby(["image_folder"])["dice/f1_score"].mean()

image_folder
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json    0.986080
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json    0.973947
Name: dice/f1_score, dtype: float64

In [24]:
matched_dice_pred_index["dice/f1_score"]

0     0.997149
1     0.994890
2     0.987024
3     0.947496
4     0.994962
        ...   
61    0.996804
62    0.998135
63         NaN
64    0.998087
65    0.923924
Name: dice/f1_score, Length: 66, dtype: float64

In [33]:
fig = px.histogram(matched_dice_pred_index["dice/f1_score"], color=matched_dice_pred_index["target_label"])
fig.update_layout(height=500, width=700)
fig.update_xaxes(title_text='Dice Coefficient')
fig.update_yaxes(title_text='Count of predictions')
fig.show()

In [34]:
matched_dice_pred_index.groupby(["image_folder"])["dice/f1_score"].mean()

image_folder
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json    0.989853
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json    0.974918
Name: dice/f1_score, dtype: float64

In [37]:
matched_dice_pred_index.groupby(["image_folder","target_label"])["dice/f1_score"].count()

image_folder                                                                                              target_label
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json  Cored           12
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json  Cored           42
                                                                                                          Diffuse          6
Name: dice/f1_score, dtype: int64

In [35]:
matched_dice_pred_index.groupby(["image_folder","target_label"])["dice/f1_score"].mean()

image_folder                                                                                              target_label
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json  Cored           0.989853
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json  Cored           0.996500
                                                                                                          Diffuse         0.823840
Name: dice/f1_score, dtype: float64

In [36]:
matched_dice_pred_index.groupby(["image_folder","target_label"])["dice/f1_score"].median()

image_folder                                                                                              target_label
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE11-039_1_AmyB_1.json  Cored           0.995304
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/val1/XE15-039_1_AmyB_1.json  Cored           0.997816
                                                                                                          Diffuse         0.843590
Name: dice/f1_score, dtype: float64